In [1]:
import json
import csv
import pandas as pd
import numpy as np
newDF = pd.DataFrame() #creates a new dataframe that's empty


In [2]:
df_gsdtsr = pd.read_csv('gsdtsr.csv', sep=';',)
df_gsdtsr.shape

(1260618, 7)

In [3]:
df_gsdtsr.head()

,Id,Name,Duration,LastRun,LastResults,Verdict,Cycle
0,1,1,27752,2016-01-02 00:00:02,[],False,1
1,2584,5,30112,2016-01-02 00:00:02,[],False,1
2,1954,4,46384,2016-01-02 00:00:02,[],False,1
3,1309,3,54264,2016-01-02 00:00:02,[],False,1
4,655,2,48238,2016-01-02 00:00:02,[],False,1


In [4]:
df_gsdtsr['Name'].nunique()


5555

In [5]:
df_gsdtsr['Verdict'].value_counts()


False    1257408
True        3210
Name: Verdict, dtype: int64

In [6]:
# Removing emply rows based on LastResults

df2=df_gsdtsr[df_gsdtsr.LastResults.str.contains('False') | df_gsdtsr.LastResults.str.contains("True")]

In [7]:
# Changing False to 0 and True to 1
df2.LastResults = df2.LastResults.str.replace(r'False', str(0), regex=True)
df2.LastResults = df2.LastResults.str.replace(r'True', str(1), regex=True)


/home/aizaz/anaconda3/envs/tensorflow/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [8]:
# Changing False to 0 and True to 1

products_list = df2.Verdict.tolist()
gender = {False: 0, True:1} 

df2.Verdict = [gender[item] for item in (products_list)] 

In [9]:
df2=df2.reset_index()

In [10]:
df2.to_csv('gsdtsr_updated'+'.csv',sep=';', index = False)

In [12]:
df_gsdtsr = pd.read_csv('gsdtsr_updated.csv',sep=';')
    

In [13]:
df_gsdtsr.head()

,index,Id,Name,Duration,LastRun,LastResults,Verdict,Cycle
0,377,66004,159,38205,2016-01-02 00:09:36,[0],0,1
1,490,203640,484,61402,2016-01-02 00:23:12,[0],0,1
2,498,8091,21,43077,2016-01-02 00:26:23,[0],0,1
3,499,78574,196,1341,2016-01-02 00:26:37,[0],0,1
4,500,38273,97,1122,2016-01-02 00:26:44,[0],0,1


In [14]:
# Dropping rows containing LastResults less than size of 3 executions

MF = df_gsdtsr.loc[:,'LastResults']                                               
to_drop = []
i=0
for a in MF:
    
    #print (a[0])
    
    
    s=''.join(x for x in a if x.isdigit())
    #print (a , " ", len(s))
    if len(s) < 3:
        
        df_gsdtsr.drop(i , inplace=True)
        
        
    i=i+1
    if (i==1000000):
        print ("Yes")

Yes


In [15]:

df_gsdtsr.head()

,index,Id,Name,Duration,LastRun,LastResults,Verdict,Cycle
175,964,66006,159,37025,2016-01-02 00:57:12,"[0, 0, 0]",0,1
451,2916,203642,484,44494,2016-01-02 01:04:20,"[0, 0, 0]",0,2
624,3342,19900,50,2408,2016-01-02 01:04:47,"[0, 0, 0]",0,2
626,3345,31913,79,13584,2016-01-02 01:04:48,"[0, 0, 0]",0,2
627,3348,48496,116,16270,2016-01-02 01:04:49,"[0, 0, 0]",0,2


In [16]:
# Adding List values of LastResults into separate E1,E2,E3 columns        
MF = df_gsdtsr.loc[:,'LastResults'].tolist()                                                           

rows = len(MF)

E1 = []
E2 = []

E3 = []
 
    
i=0
for a in MF:
    
    
    
    s=''.join(x for x in a if x.isdigit())

        

    E1.append(s[0])
    E2.append(s[1])
    E3.append(s[2])
    #print (s[0],"-",s[1],"-",s[2])

    
    
    
    
    
    i=i+1
    #print ("---")

In [17]:
df_gsdtsr['E1'] = E1
df_gsdtsr['E2'] = E2
df_gsdtsr['E3'] = E3


In [18]:
#function to reverse each row of the matrix
def reverse_row_element(array_data):
    list_data = array_data.tolist()
    list_data.reverse()
    return list_data
df_gsdtsr['E1'] =df_gsdtsr['E1'].astype(int)
df_gsdtsr['E2'] =df_gsdtsr['E2'].astype(int)
df_gsdtsr['E3'] =df_gsdtsr['E3'].astype(int)

# defining failure matrix MF
MF = df_gsdtsr.loc[:,'E1':'E3'].values

rows = len(MF)
columns = len(MF[0])

#calculating distance of failure for each test cases
distances_list = []

for i in range(rows):
    reversed_element = reverse_row_element(MF[i])
    if -1 in reversed_element:        
        for j in range(len(reversed_element)):
            if reversed_element[j] == -1:
                distances_list.insert(i, j+1)
                break
    else:
        #If distance == 0, means the test case has always been successful
        distances_list.insert(i, 0)

In [19]:
#calculating number of change of state of Pass/Fail for each test cases
change_in_status = []

for i in range(rows):
    each_element = MF[i]
    counter = 0
    for j in range(len(each_element)-1):
        if each_element[j]!=each_element[j+1]:
            counter+=1
    change_in_status.append(counter)

In [29]:
#Preparing weights based on DeepOrder algorithm
weights = np.array([0.1,0.2,0.7])
print (weights.shape)
#Extracting average execution time from dataset
df_gsdtsr['Duration'] =df_gsdtsr['Duration']

Te = df_gsdtsr.Duration.values
print (MF.shape)
#Calculating priority value for each test case
priority_value = (MF*weights).sum(axis=1)
priority_value+= np.max(Te)/Te

(3,)
(1243991, 3)


In [30]:
df_gsdtsr['DIST']=distances_list
df_gsdtsr['CHANGE_IN_STATUS']=change_in_status
df_gsdtsr['PRIORITY_VALUE'] = priority_value
df_gsdtsr.to_csv('gsdtsr_updated'+'.csv', index = False)

In [ ]:
# Update dates in LastRun according to each cycle
from datetime import timedelta


from datetime import datetime
Month_of_thirty_days = [2,4,6,9,11]
from dateutil.relativedelta import relativedelta

currentDay = datetime.now().day
currentMonth = datetime.now().month
currentYear = datetime.now().year


cycle = df_gsdtsr.loc[:,'Cycle'].values
Dates = df_gsdtsr.loc[:,'LastRun'].values

previous = 0
rows = len(cycle)
count=0
firsttime = False


# for rest of the cycle
for i in range(rows):
    each_element =cycle[i]
    #print ("Cycle :",each_element)
    #print (Dates[i])
    if i==0:
        #print ("nothing")
        datestring = Dates[i]
        dt = datetime.strptime(datestring, '%Y-%m-%d %H:%M:%S')
        Dates[i] =  str(dt.year) +"-"+ str(dt.month) +"-"+ str(dt.day)
    if i >= 1:
        datestring = Dates[i]
        dt = datetime.strptime(datestring, '%Y-%m-%d %H:%M:%S')
        value = float(each_element-1)
        # print (timedelta(days=value))
        dtt =  dt + timedelta(days=value)
        Dates[i] =  str(dtt.year) +"-"+ str(dtt.month) +"-"+ str(dtt.day)
    #print (Dates[i])
    #print ('---')  
    



 

In [ ]:
#for testing purposes only
from datetime import datetime
datestring = Dates[0]
dt = datetime.strptime(datestring, '%Y-%m-%d')
#print (dt.year, dt.month, dt.day)

from datetime import date
f_date = date(dt.year, dt.month, dt.day)
l_date = date(currentYear,currentMonth,currentDay)
delta = l_date - f_date
print(delta.days)
days.append(delta.days)

In [ ]:
# Calculating difference in LastRun test cases (number of days in return)
from datetime import datetime
Month_of_thirty_days = [2,4,6,9,11]
from dateutil.relativedelta import relativedelta

currentDay = datetime.now().day
currentMonth = datetime.now().month
currentYear = datetime.now().year


days=[]
previous = 0
rows = len(Dates)
count=0
firsttime = False
'''
#for 1st Cycle
from datetime import datetime
datestring = Dates[0]
dt = datetime.strptime(datestring, '%Y-%m-%d')
#print (dt.year, dt.month, dt.day)

from datetime import date
f_date = date(dt.year, dt.month, dt.day)
l_date = date(currentYear,currentMonth,currentDay)
delta = l_date - f_date
print(delta.days)
days.append(delta.days)
'''



# for rest of the cycle
for i in range(rows):
    each_element =cycle[i]
    #print ("Cycle :",each_element)
    
    
    
    if i ==0:
        #print ("1st Cycle")
        #for 1st Cycle
        from datetime import datetime
        datestring = Dates[i]

        dt = datetime.strptime(datestring, '%Y-%m-%d')
        #print (type(dt))
    
        f_date = date(dt.year, dt.month, dt.day)
        l_date = date(currentYear,currentMonth,currentDay)
        delta = l_date - f_date
        #print (delta)
        #print (currentYear, currentMonth, currentDay, " - ", dt.year, dt.month, dt.day)

        previous = each_element
        
    if i >= 1:
        datestring = Dates[i]
        datestring_prev = Dates[i-1]
        dt = datetime.strptime(datestring, '%Y-%m-%d')
        dt_prev = datetime.strptime(datestring_prev, '%Y-%m-%d')


        f_date = date(dt.year, dt.month, dt.day)
        l_date = date(dt_prev.year, dt_prev.month, dt_prev.day)
        delta = l_date - f_date
        #print ("** ",dt_prev.year, dt_prev.month, dt_prev.day, " - ", dt.year, dt.month, dt.day)
        

    days.append(delta.days)
    
    #print ('---')  
            
    
    
    

In [ ]:
df_gsdtsr['LastRunFeature'] = days


In [ ]:
df_gsdtsr["LastRunFeature"]=((df_gsdtsr["LastRunFeature"]-df_gsdtsr["LastRunFeature"].min())/(df_gsdtsr["LastRunFeature"].max()-df_gsdtsr["LastRunFeature"].min()))*5
df_gsdtsr["DurationFeature"]=((df_gsdtsr["Duration"]-df_gsdtsr["Duration"].min())/(df_gsdtsr["Duration"].max()-df_gsdtsr["Duration"].min()))*5
df_gsdtsr["Duration"] = df_gsdtsr["Duration"].astype(int)


In [ ]:
df_gsdtsr['Id'] = range(1, 1+len(df_gsdtsr))

In [32]:

df_gsdtsr.to_csv('gsdtsr_updated'+'.csv', index = False)

In [ ]:
df_gsdtsr.head()

In [32]:
df_gsdtsr.shape

(1243991, 16)